## Install Dependencies

In [ ]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

In [ ]:
import transformers
import torch
import json

## Inialize the models

Llama 3

In [ ]:
model_id_llama3 = "unsloth/llama-3-8b-Instruct-bnb-4bit"

llama3 = transformers.pipeline(
    "text-generation",
    model=model_id_llama3,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

Gemma

In [ ]:
model_id_gemma = "google/gemma-7b-it"

gemma = transformers.pipeline(
    "text-generation",
    model=model_id_gemma,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

Mistral

In [ ]:
model_id_mistral = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

mistral = transformers.pipeline(
    "text-generation",
    model=model_id_mistral,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

## Fallacies

In [ ]:
fallacies = """
Fallacy of Credibility: Abusive Ad Hominem; Ad Populum; Appeal to False Authority;
Appeal to Nature; Appeal to Tradition; Guilt by Association; Tu Quoque.
Fallacy of Logic: Causal Oversimplification; Circular Reasoning; Equivocation;
False Analogy; False Causality; False Dilemma; Hasty Generalization; Slippery Slope; Straw Man; Fallacy of Division.
Appeal to Emotion: Appeal to Positive Emotion; Appeal to Anger; Appeal to Fear;
Appeal to Pity; Appeal to Ridicule; Appeal to Worse Problem.
"""

In [ ]:
fallacies_with_examples = """
Slippery slope:\n\nIf we ban assault rifles, next they'll ban all guns, and before you know it, we'll be living in a police state.,
Hasty generalization:\n\nI met two people from New York City, and both were rude. Therefore, all New Yorkers are rude.,
False analogy:\n\nComparing buying a car to choosing a spouse is a false analogy because the two situations are vastly different.,
Guilt by association:\n\nYou shouldn't listen to John's opinion on politics; he's friends with several criminals.,
Causal oversimplification:\n\nThe rise in crime rates is due to violent video games; therefore, banning them will solve the problem.,
Ad populum:\n\nEveryone is buying the new smartphone, so it must be the best one on the market.,
Circular reasoning:\n\nThe Bible is the word of God because God says it is in the Bible.,
Appeal to fear:\n\nIf you don't vote for me, your taxes will skyrocket and crime will run rampant.,
False causality:\n\nEvery time I wash my car, it rains. Therefore, washing my car causes rain.,
Fallacy of division:\n\nThat company is profitable, so all of its employees must be well-paid.,
Appeal to ridicule:\n\nYou shouldn't listen to Mary's argument about climate change; she believes the Earth is flat!,
Appeal to worse problems:\n\nWhy are we worrying about pollution when there are children starving in the world?,
Abusive ad hominem:\n\nYou can't trust anything Tom says; he's just a loser.,
Appeal to false authority:\n\nMy doctor says that smoking is good for you, so it must be true.,
Appeal to positive emotion:\n\nIf you care about the environment, you'll buy this eco-friendly product.,
Appeal to nature:\n\nNatural remedies are always better than pharmaceutical drugs.,
Equivocation:\n\nThe sign said 'Fine for parking here,' so I thought it would be fine to park there.,
Appeal to anger:\n\nYou wouldn't want to be a coward, would you? Then you better join the military!,
False dilemma:\n\nEither we ban all guns or we'll continue to have mass shootings.,
Straw man:\n\nOpponents of gun control argue that we should give guns to everyone, including criminals and children.,
Appeal to tradition:\n\nWe should continue to use fossil fuels because that's how we've always done it.,
Tu quoque:\n\nYou say I shouldn't smoke, but I've seen you smoke before!
"""

### Version №1: Vanilla zero-shot version of ToT prompting.

In [ ]:
def tot_prompt_1(input_text, pipeline):
    messages = [
        {"role"   : "user",
        "content":
        f"As a content reviewer, I provide multiple fallacies which might occur in the text; \
        you need to identify the falacies if there're such in the text.\
        {fallacies}\
        Text: {input_text}\
        Walk me through this context in manageable parts step by step, \
        summarizing and analyzing as we go.\
        Answer:\
        "},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
# The answer is "false analogy"
print(tot_prompt_1(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """,
                   pipeline=llama3))

Let's break down the text and identify any fallacies that might be present.

**Step 1: Understand the context**
The author is discussing the difference between an "smurf" and an "alt" in the context of online gaming.

**Step 2: Identify the claims**
The author makes two claims:

1. "Someone once told me they have an'alt' cause their main account was too high of rank to play with their friends."
2. "It's exactly the same as smurfing."

**Step 3: Analyze the claims**

1. The first claim is an anecdote, which is a personal experience or story told by someone. However, the author doesn't provide any further context or evidence to support this claim. This could be an example of the **Appeal to Authority** fallacy, where the author is relying on an unverified anecdote as evidence.
2. The second claim is a comparison between having an "alt" and "smurfing". The author is implying that having an "alt" is equivalent to "smurfing", which is a common gaming term for creating a new account to gain 

In [ ]:
# The answer is "circular reasoning"
print(tot_prompt_1(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """,
                   pipeline=gemma))

**Fallacies in the text:**

**1. Ad Populum:** The statement "America is the best place to live" is an ad populum fallacy, as it relies on the popularity of the United States to support the claim.

**2. Appeal to Tradition:** The statement "America is the best place to live" also appeals to tradition, as it is a commonly held belief in the United States.

**3. Straw Man:** The statement "America is the best place to live" does not address the potential drawbacks of living in the United States, such as high crime rates or income inequality.

**Answer:**

The text claims that America is the best place to live, based on the premise that it is better than any other country. However, this claim is fallacious due to the ad populum, appeal to tradition, and straw man fallacies discussed above. There are many factors to consider when comparing countries, and it is important to weigh the pros and cons of each place before making a judgment.


In [ ]:
# The answer is "causal oversimplification" and "false causality"
print(tot_prompt_1(input_text=
             """
             The last Democrat winner of the New Hampshire primary won the general election.
             This year, the winner of the New Hampshire primary will win the general election.
             """,
                   pipeline=llama3))

Let's break down the text and identify any potential fallacies.

**Step 1: Identify the claims**

The text consists of two claims:

1. "The last Democrat winner of the New Hampshire primary won the general election."
2. "This year, the winner of the New Hampshire primary will win the general election."

**Step 2: Analyze the first claim**

The first claim is a statement of fact. It appears to be a historical fact about a previous election. There is no apparent fallacy in this claim.

**Step 3: Analyze the second claim**

The second claim is a prediction. It is making a claim about the outcome of an upcoming election. Here, we can identify a potential fallacy:

* **Fallacy of Credibility: Hasty Generalization**: The claim is making a general statement about the outcome of the upcoming election based on a single historical example. This is a hasty generalization, as it is assuming that the outcome of the next election will be similar to the previous one without considering any other fact

### Version №2: Vanilla zero-shot version of ToT prompting with modified output.

In [ ]:
def tot_prompt_2(input_text, pipeline):
    messages = [
            {"role"   : "user",
            "content":
            f"As a content reviewer, I provide multiple fallacies which might occur in the text; \
            you need to identify the falacies if there're such in the text, else return 'no fallacy'.\
            {fallacies}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go. \
            Then, output just the parts of text with their respective identified fallacies on the last line separated by ';'.\
            Answer:\
            "},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
# The answer is "false analogy"
print(tot_prompt_2(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """,
                   pipeline=llama3))

Let's break down the text and analyze it step by step.

**Step 1: Understand the context**
The text is discussing the difference between "smurf" and "alt" in a gaming context. Someone is saying that having an "alt" (alternative account) is the same as "smurfing" (using a high-ranked account to play with friends).

**Step 2: Identify the fallacies**
I've identified the following fallacies in the text:

* **Appeal to False Authority**: The text mentions someone telling the author that having an "alt" is the same as smurfing, but it doesn't provide any evidence or credible source to support this claim.
* **False Analogy**: The author is comparing having an "alt" to smurfing, but these two concepts are not equivalent. Smurfing is a specific behavior (using a high-ranked account to play with friends), while having an "alt" is a different thing (using an alternative account).

**Output**
Here are the parts of the text with their respective identified fallacies:

POST: Someone once told me th

In [ ]:
# The answer is "circular reasoning"
print(tot_prompt_2(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """,
                   pipeline=gemma))

**Answer:**

The text is:

America is the best place to live, because it's better than any other country.

**Fallacies:**

- Ad Populum
- Appeal to Positive Emotion

**Text with identified fallacies:**

America is the best place to live, because it's better than any other country.

**Fallacies:**

Ad Populum; Appeal to Positive Emotion


In [ ]:
# The answer is "causal oversimplification" and "false causality"
print(tot_prompt_2(input_text=
             """
             The last Democrat winner of the New Hampshire primary won the general election.
             This year, the winner of the New Hampshire primary will win the general election.
             """,
                   pipeline=llama3))

Let's break down the text step by step.

**Step 1:** The text consists of two sentences.

1. "The last Democrat winner of the New Hampshire primary won the general election."
2. "This year, the winner of the New Hampshire primary will win the general election."

**Step 2:** Analyze the first sentence:

* The sentence states a fact about the past, which is a neutral statement.
* It does not contain any logical fallacies.

**Step 3:** Analyze the second sentence:

* The sentence makes a claim about the future based on the past.
* This is an example of **Appeal to Authority**, as it implies that the past is a reliable indicator of the future. This is a fallacy of credibility.

**Step 4:** Combine the two sentences:

The text is making a claim about the future based on a past event, and using that past event as a basis for predicting the future.

**Output:**

The text contains the fallacy of **Appeal to Authority**.

Text with fallacies: "This year, the winner of the New Hampshire primary 

In [ ]:
# The answer is "slippery slope"
print(tot_prompt_2(input_text=
             """
             TITLE: Gatineau police officer tests positive, but masks still not mandatory at checkpoints POST:
             Guess what? police need to interact with people at close quarters. they often need to touch or grapple.
             The masks do little. Cool it. Once they wear masks your anxiety will force you to demand the next step.
             biohazard suits? get real.
             """,
                   pipeline=gemma))

**Answer:**

The text contains the following fallacies:

- Appeal to Authority
- Appeal to False Authority
- Hasty Generalization
- Straw Man

**Text:**

Gatineau police officer tests positive, but masks still not mandatory at checkpoints.

Guess what? police need to interact with people at close quarters. They often need to touch or grapple.

The masks do little. Cool it. Once they wear masks your anxiety will force you to demand the next step.

biohazard suits? get real.


In [ ]:
# The answer is "appeal to nature"
print(tot_prompt_2(input_text=
             """
             TITLE: Sweden sees higher coronavirus death rate than US after refusing lockdown POST:
             I wonder how is New York planning on getting out of the lockdown,
             because when they get out of lockdown probably some cases are going to be going
             on and the whole infection thing will start again. POST: You cant stop nature.
             Youre going to be infected if you havent already. We are always trying to fight nature.
             We just have to let it run its course.
             """,
                   pipeline=llama3))

Let's break down the text step by step:

**POST 1:** "I wonder how is New York planning on getting out of the lockdown, because when they get out of lockdown probably some cases are going to be going on and the whole infection thing will start again."

* This post seems to be expressing concern about the effectiveness of the lockdown in New York.
* The fallacy present here is: **False Dilemma** (assuming that there are only two options: lockdown or no lockdown, when in reality there might be other alternatives).

**POST 2:** "You can't stop nature. You're going to be infected if you haven't already. We are always trying to fight nature. We just have to let it run its course."

* This post seems to be arguing that the coronavirus is a natural phenomenon that cannot be stopped, and that trying to fight it is futile.
* The fallacies present here are: **Appeal to Nature** (using the concept of "nature" to justify a conclusion without providing evidence) and **Appeal to False Authority** (a

In [ ]:
# The answer is "no fallacy"
print(tot_prompt_2(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """,
                   pipeline=gemma))

**Walk-through:**

**Summary:**

The text claims that President Putin's sea-ice loss is his gain, arguing that it will provide him with access to shipping routes, energy reserves, and a strategic military advantage.

**Analysis:**

**Fallacies:**

* **Appeal to Positive Emotion:** The text appeals to the reader's positive emotions, such as patriotism and national pride, by highlighting Russia's status as the largest country on the planet.
* **Appeal to Nature:** The text appeals to the natural phenomenon of sea-ice loss, claiming that it will benefit Russia's shipping routes and energy reserves.
* **Causal Oversimplification:** The text oversimplifies the relationship between sea-ice loss and its impact on shipping routes and energy reserves.

**Text with identified fallacies:**

The sea-ice loss is President Vladimir Putin's gain; Already the largest country on the planet, Russia stands to gain access to shipping routes and energy reserves, and a strategic military advantage from the 

### Version №3: Vanilla few-shot version of ToT prompting.

In [ ]:
def tot_prompt_3(input_text, pipeline):
    messages = [
            {"role"   : "user",
            "content":
            f"As a content reviewer, I provide multiple fallacies and their examples which might occur in the text; \
            you need to identify the falacies if there're such in the text, else return 'no fallacy'.\
            {fallacies_with_examples}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go.\
            Then, provide only the parts of text with their respective identified fallacies separated by ';' or the 'no fallacy' label.\
            Answer:\
            "},
    ]
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
# The answer is "false analogy"
print(tot_prompt_3(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """,
                   pipeline=llama3))

Let's break down the text into manageable parts and analyze it step by step.

**Part 1:** TITLE: There is a difference between a'smurf' and an 'alt'.

* No fallacy detected in this part.

**Part 2:** POST: Someone once told me they have an 'alt' cause their main account was too high of rank to play with their friends. It's exactly the same as smurfing.

* **Appeal to false analogy:** The author is comparing having an 'alt' account to smurfing, which are not the same thing. Smurfing is a practice of creating multiple accounts to play with friends, while having an 'alt' account is a common practice in online gaming where a player creates an alternative account to play with friends who are not in their main account's rank.

Here are the parts of the text with their respective identified fallacies separated by ';':

* No fallacy; Appeal to false analogy


In [ ]:
# The answer is "circular reasoning"
print(tot_prompt_3(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """,
                   pipeline=gemma))

**Answer:**

America is the best place to live, because it's better than any other country.

**Fallacies:**

- Appeal to tradition
- Straw man
- False dilemma

**No fallacy:**

No fallacy


In [ ]:
# The answer is "causal oversimplification" and "false causality"
print(tot_prompt_3(input_text=
             """
             The last Democrat winner of the New Hampshire primary won the general election.
             This year, the winner of the New Hampshire primary will win the general election.
             """,
                   pipeline=llama3))

Let's break down the text step by step.

**Step 1: Identify the claims**
The text makes two claims:

1. The last Democrat winner of the New Hampshire primary won the general election.
2. This year, the winner of the New Hampshire primary will win the general election.

**Step 2: Analyze the claims**
The first claim is a historical fact, stating that a previous winner of the New Hampshire primary won the general election. The second claim is a prediction, stating that the winner of the current New Hampshire primary will also win the general election.

**Step 3: Identify the fallacy (if any)**
The fallacy in this text is:

**Hasty generalization**:
The second claim is making a generalization based on a single data point (the previous winner). There is no evidence to support the claim that the winner of the current New Hampshire primary will also win the general election. This is a hasty generalization, as the outcome of the current election is uncertain and may not follow the same patter

In [ ]:
# The answer is "slippery slope"
print(tot_prompt_3(input_text=
             """
             TITLE: Gatineau police officer tests positive, but masks still not mandatory at checkpoints POST:
             Guess what? police need to interact with people at close quarters. they often need to touch or grapple.
             The masks do little. Cool it. Once they wear masks your anxiety will force you to demand the next step.
             biohazard suits? get real.
             """,
                   pipeline=gemma))

**Answer:**

The text has the following fallacies:

Hasty generalization;
Causal oversimplification;
Fallacy of division;
Appeal to anger;
False dilemma;
Straw man;
Appeal to tradition;
Tu quoque

**No fallacy:**

No fallacy found in the text.


In [ ]:
# The answer is "appeal to nature"
print(tot_prompt_3(input_text=
             """
             TITLE: Sweden sees higher coronavirus death rate than US after refusing lockdown POST:
             I wonder how is New York planning on getting out of the lockdown,
             because when they get out of lockdown probably some cases are going to be going
             on and the whole infection thing will start again. POST: You cant stop nature.
             Youre going to be infected if you havent already. We are always trying to fight nature.
             We just have to let it run its course.
             """,
                   pipeline=llama3))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Let's break down the text step by step.

**Initial Post**
I wonder how is New York planning on getting out of the lockdown, because when they get out of lockdown probably some cases are going to be going on and the whole infection thing will start again.

* The speaker is wondering about New York's plan to exit the lockdown, and is concerned that cases will continue to arise even after the lockdown is lifted.

**Response Post**
You can't stop nature. You're going to be infected if you haven't already. We are always trying to fight nature. We just have to let it run its course.

* The respondent is arguing that it's futile to try to stop the spread of the virus, and that nature will ultimately prevail. They're suggesting that the best course of action is to let the virus run its course.

Now, let's identify any fallacies in the text:

* No fallacy in the initial post.
* The respondent's statement "You can't stop nature" is an example of **Appeal to nature**, which is the idea that natur

In [ ]:
# The answer is "no fallacy"
print(tot_prompt_3(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """,
                   pipeline=gemma))

**Answer:**

The text does not contain any fallacies. Therefore, the answer is 'no fallacy'.

**Text:**

The sea-ice loss is President Vladimir Putin's gain. Already the largest country on the planet, Russia stands to gain access to shipping routes and energy reserves, and a strategic military advantage from the opening of the Arctic. Along the Russian coastline, which makes up more than half the Arctic total, winds and currents push old ice away from potential shipping lanes and prevent the build-up of thicker, multi-year ice that would leave other parts of the Arctic impassable for longer periods.

**Fallacies:**

No fallacies.


### Version №4: Advanced zero-shot version of ToT prompting: two-step prompting using reasoning + refining conclusion.

In [ ]:
def tot_prompt_4(input_text, pipeline):
    # Step 1: Initial reasoning
    messages = [
        {"role": "user", "content":
            f"As a content reviewer, I provide multiple fallacies which might occur in the text; \
            you need to identify the fallacies if there're such in the text, else return 'no fallacy'.\
            {fallacies}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go. \
            Answer:\
            "}
    ]

    # Generate the initial analysis using the pipeline
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    initial_analysis = outputs[0]["generated_text"][len(prompt):]

    # Step 2: Refining the conclusion
    messages_refine = [
        {"role": "user", "content":
            f"Based on the analysis provided, distill the findings into a concise summary. \
            For each identified fallacy, specify the text segment and the corresponding fallacy type. \
            Summarize the identified fallacies and their types from the initial analysis:\
            {initial_analysis}\
            Provide only the text where the fallacy was found with their respective fallacies in the format \
            {{text n:fallacy n}},{{text m:fallacy m}}\
            or the 'no fallacy' label if no fallacies in the whole text were detected.\
            Do not output any other text or comments."
        }
    ]

    prompt_refine = pipeline.tokenizer.apply_chat_template(
        messages_refine,
        tokenize=False,
        add_generation_prompt=True
    )

    outputs_refine = pipeline(
        prompt_refine,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    refined_output = outputs_refine[0]["generated_text"][len(prompt_refine):]

    return refined_output

In [ ]:
# The answer is "false analogy"
print(tot_prompt_4(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """,
                   pipeline=llama3))

Here is the concise summary:

{Someone once told me...: Appeal to False Authority},{Playing with friends due to high rank, to smurfing: False Analogy},{The reader should learn and stop using the terms interchangeably: Appeal to Emotion}


In [ ]:
# The answer is "circular reasoning"
print(tot_prompt_4(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """,
                   pipeline=gemma))

**Summary:**

The text contains the fallacies of ad populum, oversimplification, and straw man.

**Fallacies:**

* **Fallacy of Ad Populum:** "America is presented as the best place to live based on the assumption that it is better than any other country."
* **Fallacy of Oversimplification:** "America is the best place to live" is an oversimplification of a complex issue.
* **Fallacy of Straw Man:** The argument that America is the best place to live assumes that other countries are not as good.

**No other fallacies were detected.**


In [ ]:
# The answer is "no fallacy"
print(tot_prompt_4(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """,
                   pipeline=llama3))

### Version №5: Advanced few-shot version of ToT prompting: two-step prompting using reasoning + refining conclusion.

In [ ]:
def tot_prompt_5(input_text, pipeline):
    # Step 1: Initial reasoning
    messages = [
        {"role": "user", "content":
            f"As a content reviewer, I provide multiple fallacies and their examples which might occur in the text; \
            you need to identify the fallacies if there're such in the text, else return 'no fallacy'. Please also return the original text in the end.\
            {fallacies_with_examples}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go. \
            Answer:\
            "}
    ]

    # Generate the initial analysis using the pipeline
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
    )

    initial_analysis = outputs[0]["generated_text"][len(prompt):]

    # Step 2: Refining the conclusion
    messages_refine = [
        {"role": "user", "content":
            f"For each identified fallacy, specify the text segment and the corresponding fallacy type:\
            {initial_analysis}\
            Provide only the fallacies and copy the piece from the original texts containing these fallacies\
             in the following format (where keys are the fallacies, values are the slices of the original text where the fallacies were detected): \
            {{fallacy type X: text X}}\
            or an empty dictionary if no fallacies in the whole text were detected.\
            Please do not give any comments and do not paraphrase anything."
        }
    ]

    prompt_refine = pipeline.tokenizer.apply_chat_template(
        messages_refine,
        tokenize=False,
        add_generation_prompt=True
    )

    outputs_refine = pipeline(
        prompt_refine,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
    )

    refined_output = outputs_refine[0]["generated_text"][len(prompt_refine):]

    return refined_output

In [ ]:
# The answer is "false analogy"
output = tot_prompt_5(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """,
                   pipeline=llama3)

In [ ]:
output

"Here are the fallacies with the corresponding text segments:\n\n{False analogy: Someone once told me they have an 'alt' cause their main account was too high of rank to play with their friends. It's exactly the same as smurfing.}\n{Equivocation: Someone once told me they have an 'alt' cause their main account was too high of rank to play with their friends. It's exactly the same as smurfing.}\n{Appeal to authority: Someone once told me they have an 'alt' cause their main account was too high of rank to play with their friends. It's exactly the same as smurfing.}"

In [ ]:
# The answer is "circular reasoning"
output2 = tot_prompt_5(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """,
                   pipeline=llama3)

In [ ]:
output2

'{False comparison: "America is the best place to live, because it\'s better than any other country."}'

In [ ]:
# The answer is "no fallacy"
output3 = tot_prompt_5(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """,
                   pipeline=llama3)

In [ ]:
output3

'{Appeal to authority: "The sea-ice loss is President Vladimir Putin\'s gain. Already the largest country on the planet, Russia stands to gain access to shipping routes and energy reserves, and a strategic military advantage from the opening of the Arctic."}'

## Extract all fallacies for all the data

In [ ]:
f = open('/content/Test_Dataset.json')
data = json.load(f)

In [ ]:
for n, sample in enumerate(data[100:150]):
  print(n)
  for k in sample:
    output = tot_prompt_5(input_text=k,
                          pipeline=llama3)
    sample[k] = output

In [ ]:
with open('outputs_100_150.json', 'w') as f:
    json.dump(data[:50], f)